<a href="https://colab.research.google.com/github/ShoaibSheriff/Computer-Vision/blob/master/Direction_AVA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt

import torch.optim as optim
from torch.autograd import Variable
import torchvision.transforms as transforms


/home/vaibhav/.local/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


# Direction

In [2]:
import torchvision.models as models
resnet34 = models.resnet34(pretrained=True)

In [3]:
import torch
import torch.nn as nn

resnet_binary = nn.Sequential(resnet34, nn.Sequential(nn.Linear(1000,2), nn.Softmax()))

In [4]:
import os
path1  = 'allimages/left/'
ls1 = os.listdir(path1)
images = []
labels = []
import scipy.misc as sm

for  i in ls1:
    labels.append(0)
    frma1 = sm.imread(path1 + i)
    frame1 =sm.imresize(frma1, (224,224))
    
    images.append(frame1)

path1  = 'allimages/right/'
ls1 = os.listdir(path1)

for  i in ls1:
    labels.append(1)
    frma1 = sm.imread(path1 + i)
    frame1 =sm.imresize(frma1, (224,224))
    
    images.append(frame1)
    


/home/vaibhav/.local/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  # Remove the CWD from sys.path while we load stuff.
/home/vaibhav/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  # This is added back by InteractiveShellApp.init_path()
/home/vaibhav/.local/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/home/vaibhav/.local/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transfor

In [5]:
videos = images


In [6]:
from torch.utils.data import DataLoader,sampler,Dataset

class ActionSearchDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, action, act_training_data):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        print('init')

    def __len__(self):

        return len(videos)

    def __getitem__(self, idx):
      
        frame = videos[idx]
        
        train_transforms = transforms.Compose([
          transforms.ToTensor(), 
          transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
        
        frame = train_transforms(frame)
        label = labels[idx]
        sample = {'image': frame, 'label': label}
        return sample

In [7]:
as_dataset=ActionSearchDataset(action = 4, act_training_data = 2)

for i in range(10):
    sample=as_dataset[i]
    print(sample['image'].shape)
    print(sample['label'])

init
torch.Size([3, 224, 224])
0
torch.Size([3, 224, 224])
0
torch.Size([3, 224, 224])
0
torch.Size([3, 224, 224])
0
torch.Size([3, 224, 224])
0
torch.Size([3, 224, 224])
0
torch.Size([3, 224, 224])
0
torch.Size([3, 224, 224])
0
torch.Size([3, 224, 224])
0
torch.Size([3, 224, 224])
0


In [8]:
as_dataset_batches = DataLoader(as_dataset, batch_size=10,
                        shuffle=True, num_workers=1)
for i,sample in enumerate(as_dataset_batches):
   
    print(i,sample['image'].shape, sample['label'])
    if i>10: 
        break

0 torch.Size([10, 3, 224, 224]) tensor([1, 0, 1, 1, 0, 0, 0, 0, 1, 0])
1 torch.Size([10, 3, 224, 224]) tensor([0, 1, 1, 1, 0, 0, 0, 0, 1, 1])
2 torch.Size([10, 3, 224, 224]) tensor([0, 0, 1, 0, 0, 1, 1, 1, 0, 1])
3 torch.Size([10, 3, 224, 224]) tensor([1, 0, 1, 0, 1, 1, 1, 0, 0, 1])
4 torch.Size([10, 3, 224, 224]) tensor([0, 0, 0, 1, 1, 0, 0, 1, 0, 0])
5 torch.Size([10, 3, 224, 224]) tensor([1, 0, 0, 0, 0, 0, 0, 0, 1, 1])
6 torch.Size([10, 3, 224, 224]) tensor([0, 1, 0, 0, 1, 0, 1, 1, 1, 0])
7 torch.Size([10, 3, 224, 224]) tensor([1, 1, 1, 1, 0, 0, 0, 1, 0, 1])
8 torch.Size([10, 3, 224, 224]) tensor([1, 0, 0, 0, 1, 0, 1, 0, 1, 0])
9 torch.Size([10, 3, 224, 224]) tensor([0, 0, 0, 1, 1, 0, 0, 0, 1, 1])
10 torch.Size([10, 3, 224, 224]) tensor([1, 0, 0, 1, 1, 1, 1, 0, 1, 0])
11 torch.Size([10, 3, 224, 224]) tensor([0, 0, 0, 0, 1, 0, 0, 0, 1, 0])


In [9]:
def train(model, loss_fn, optimizer, dataloader, num_epochs = 1):
    for epoch in range(num_epochs):
        print('Starting epoch %d / %d' % (epoch + 1, num_epochs))
        model.train()
        loss1 = 0.0
        for t, sample in enumerate(dataloader):
            x_var = Variable(sample['image'].float().cuda())
            y_var = Variable(sample['label'].long().cuda())

            scores = model(x_var)
            loss =loss_fn(scores, y_var)
            loss1 += loss.data
            if (t + 1) % 100 == 0:
                print('t = %d, loss = %.4f' % (t + 1, loss1))
                loss1 = 0
        
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        check_accuracy(resnet_binary, as_dataset_batches)


In [10]:
import copy
gpu_dtype = torch.cuda.FloatTensor

resnet_binary = copy.deepcopy(resnet_binary)
resnet_binary.cuda()

Sequential(
  (0): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tr

In [ ]:
# This sets the model in "training" mode. 
# This is relevant for some layers that may have different behavior
# in training mode vs testing mode, such as Dropout and BatchNorm. 
loss_fn = nn.CrossEntropyLoss().cuda()
optimizer = optim.Adam(resnet_binary.parameters(), lr = 1e-4)

torch.cuda.random.manual_seed(12345)
resnet_binary.train() 
train(resnet_binary, loss_fn, optimizer, as_dataset_batches, num_epochs=200) 

Starting epoch 1 / 200


/home/vaibhav/.local/lib/python3.6/site-packages/torch/nn/modules/container.py:91: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


t = 100, loss = 51.2214
t = 200, loss = 51.4405
t = 300, loss = 51.7261
t = 400, loss = 47.5438
t = 500, loss = 50.5726
t = 600, loss = 52.2663
t = 700, loss = 53.1925
t = 800, loss = 49.7630
t = 900, loss = 49.4827
t = 1000, loss = 51.3456
t = 1100, loss = 51.4319
t = 1200, loss = 51.6573
t = 1300, loss = 55.5573
t = 1400, loss = 52.5126
t = 1500, loss = 51.6348
t = 1600, loss = 52.0572
t = 1700, loss = 48.7023
t = 1800, loss = 51.4707
t = 1900, loss = 51.8914
t = 2000, loss = 49.5288
Got 16420 / 20541 correct (79.94)
Starting epoch 2 / 200
t = 100, loss = 53.4346
t = 200, loss = 52.9687
t = 300, loss = 51.8831
t = 400, loss = 49.4158
t = 500, loss = 47.9329
t = 600, loss = 48.3467
t = 700, loss = 49.5762
t = 800, loss = 52.0965
t = 900, loss = 50.3110
t = 1000, loss = 48.6520
t = 1100, loss = 49.9740
t = 1200, loss = 48.6794
t = 1300, loss = 48.1832
t = 1400, loss = 48.3802
t = 1500, loss = 48.3113
t = 1600, loss = 47.5981
t = 1700, loss = 46.5966
t = 1800, loss = 47.0387
t = 1900, l

t = 100, loss = 42.5749
t = 200, loss = 42.2261
t = 300, loss = 44.0995
t = 400, loss = 43.2445
t = 500, loss = 43.2704
t = 600, loss = 44.2948
t = 700, loss = 42.8409
t = 800, loss = 41.8590
t = 900, loss = 44.1677
t = 1000, loss = 44.4067
t = 1100, loss = 42.4297
t = 1200, loss = 42.3270
t = 1300, loss = 43.0767
t = 1400, loss = 46.2186
t = 1500, loss = 43.2009
t = 1600, loss = 45.6957
t = 1700, loss = 43.6336
t = 1800, loss = 41.9269
t = 1900, loss = 43.2559
t = 2000, loss = 43.6693
Got 18069 / 20541 correct (87.97)
Starting epoch 17 / 200
t = 100, loss = 42.8186
t = 200, loss = 44.3881
t = 300, loss = 44.1011
t = 400, loss = 42.7260
t = 500, loss = 44.7734
t = 600, loss = 44.0008
t = 700, loss = 44.9511
t = 800, loss = 44.0000
t = 900, loss = 43.8436
t = 1000, loss = 43.3458
t = 1100, loss = 42.9832
t = 1200, loss = 42.8490
t = 1300, loss = 43.1359
t = 1400, loss = 42.7252
t = 1500, loss = 45.4848
t = 1600, loss = 42.6428
t = 1700, loss = 43.5682
t = 1800, loss = 41.3119
t = 1900, 

t = 100, loss = 41.8690
t = 200, loss = 40.9287
t = 300, loss = 43.5027
t = 400, loss = 41.0122
t = 500, loss = 42.0299
t = 600, loss = 41.6507
t = 700, loss = 40.3333
t = 800, loss = 40.9189
t = 900, loss = 41.1962
t = 1000, loss = 40.6685
t = 1100, loss = 40.6785
t = 1200, loss = 40.5501
t = 1300, loss = 39.4097
t = 1400, loss = 41.6922
t = 1500, loss = 40.4098
t = 1600, loss = 43.8143
t = 1700, loss = 41.6567
t = 1800, loss = 42.0560
t = 1900, loss = 41.3260
t = 2000, loss = 42.5702
Got 18431 / 20541 correct (89.73)
Starting epoch 32 / 200
t = 100, loss = 41.5883
t = 200, loss = 42.2851
t = 300, loss = 42.4759
t = 400, loss = 42.3346
t = 500, loss = 41.2996
t = 600, loss = 40.0859
t = 700, loss = 41.3109
t = 800, loss = 40.4039
t = 900, loss = 41.9999
t = 1000, loss = 41.4476
t = 1100, loss = 42.3344
t = 1200, loss = 39.5171
t = 1300, loss = 40.3222
t = 1400, loss = 41.0528
t = 1500, loss = 41.2789
t = 1600, loss = 42.0036
t = 1700, loss = 40.5061
t = 1800, loss = 43.5281
t = 1900, 

...

t = 100, loss = 39.8895
t = 200, loss = 40.5684
t = 300, loss = 38.6975
t = 400, loss = 40.3327
t = 500, loss = 39.6070
t = 600, loss = 40.4497
t = 700, loss = 39.4786
t = 800, loss = 39.9592
t = 900, loss = 39.3815
t = 1000, loss = 39.6137
t = 1100, loss = 39.4501
t = 1200, loss = 38.8294
t = 1300, loss = 39.6259
t = 1400, loss = 38.9090
t = 1500, loss = 40.2223
t = 1600, loss = 41.2561
t = 1700, loss = 40.6124
t = 1800, loss = 40.5948
t = 1900, loss = 39.9599
t = 2000, loss = 40.4927
Got 18813 / 20541 correct (91.59)
Starting epoch 62 / 200
t = 100, loss = 40.3833
t = 200, loss = 41.4363
t = 300, loss = 40.5453
t = 400, loss = 39.9721
t = 500, loss = 39.2774
t = 600, loss = 40.1922
t = 700, loss = 39.3557
t = 800, loss = 39.2391
t = 900, loss = 39.6182
t = 1000, loss = 39.6095
t = 1100, loss = 38.5760
t = 1200, loss = 38.7699
t = 1300, loss = 40.9560
t = 1400, loss = 39.4825
t = 1500, loss = 41.8288
t = 1600, loss = 41.3763
t = 1700, loss = 40.5982
t = 1800, loss = 40.9525
t = 1900, 

t = 100, loss = 38.0822
t = 200, loss = 38.0804
t = 300, loss = 40.3021
t = 400, loss = 40.9841
t = 500, loss = 40.1367
t = 600, loss = 41.3003
t = 700, loss = 40.7767
t = 800, loss = 40.7066
t = 900, loss = 40.9284
t = 1000, loss = 38.7080
t = 1100, loss = 39.1899
t = 1200, loss = 41.3941
t = 1300, loss = 41.1528
t = 1400, loss = 38.7549
t = 1500, loss = 41.4310
t = 1600, loss = 39.1280
t = 1700, loss = 40.7081
t = 1800, loss = 39.6668
t = 1900, loss = 38.6499
t = 2000, loss = 40.7971
Got 18711 / 20541 correct (91.09)
Starting epoch 77 / 200
t = 100, loss = 41.1385
t = 200, loss = 39.2701
t = 300, loss = 40.1362
t = 400, loss = 39.3851
t = 500, loss = 40.0722
t = 600, loss = 40.4499
t = 700, loss = 40.8416
t = 800, loss = 39.8120
t = 900, loss = 41.6072
t = 1000, loss = 40.2027
t = 1100, loss = 39.3099
t = 1200, loss = 39.2593
t = 1300, loss = 40.4878
t = 1400, loss = 40.0020
t = 1500, loss = 40.7144
t = 1600, loss = 39.8376
t = 1700, loss = 39.8469
t = 1800, loss = 40.8990
t = 1900, 

t = 100, loss = 39.7044
t = 200, loss = 39.9398
t = 300, loss = 40.0704
t = 400, loss = 38.2802
t = 500, loss = 39.3878
t = 600, loss = 39.7308
t = 700, loss = 40.6132
t = 800, loss = 40.2115
t = 900, loss = 40.9252
t = 1000, loss = 40.0265
t = 1100, loss = 41.0461
t = 1200, loss = 40.8906
t = 1300, loss = 39.3198
t = 1400, loss = 40.9255
t = 1500, loss = 40.9379
t = 1600, loss = 40.2116
t = 1700, loss = 40.3853
t = 1800, loss = 41.1283
t = 1900, loss = 38.9052
t = 2000, loss = 40.0876
Got 18766 / 20541 correct (91.36)
Starting epoch 92 / 200
t = 100, loss = 39.7132
t = 200, loss = 40.9431
t = 300, loss = 41.1342
t = 400, loss = 40.7524
t = 500, loss = 38.8388
t = 600, loss = 38.7190
t = 700, loss = 40.1585
t = 800, loss = 39.8143
t = 900, loss = 40.0178
t = 1000, loss = 41.1297
t = 1100, loss = 39.8673
t = 1200, loss = 40.2463
t = 1300, loss = 40.3955
t = 1400, loss = 39.2233
t = 1500, loss = 41.1483
t = 1600, loss = 40.5687
t = 1700, loss = 40.9953
t = 1800, loss = 40.4127
t = 1900, 

t = 100, loss = 41.1403
t = 200, loss = 41.5348
t = 300, loss = 39.3530
t = 400, loss = 39.5386
t = 500, loss = 41.0654
t = 600, loss = 39.6455
t = 700, loss = 39.8225
t = 800, loss = 38.8623
t = 900, loss = 39.8616
t = 1000, loss = 40.4061
t = 1100, loss = 39.5458
t = 1200, loss = 39.4735
t = 1300, loss = 37.5799
t = 1400, loss = 39.2282
t = 1500, loss = 39.0794
t = 1600, loss = 41.0648
t = 1700, loss = 40.4333
t = 1800, loss = 39.9884
t = 1900, loss = 40.6086
t = 2000, loss = 37.7414
Got 18868 / 20541 correct (91.86)
Starting epoch 107 / 200
t = 100, loss = 39.5229
t = 200, loss = 40.0386
t = 300, loss = 39.8473
t = 400, loss = 39.5734
t = 500, loss = 38.1870
t = 600, loss = 39.7618
t = 700, loss = 39.9377
t = 800, loss = 40.2031
t = 900, loss = 41.0244
t = 1000, loss = 37.7825
t = 1100, loss = 39.9028
t = 1200, loss = 39.1172
t = 1300, loss = 38.8264
t = 1400, loss = 39.9389
t = 1500, loss = 39.6987
t = 1600, loss = 40.3484
t = 1700, loss = 42.6122
t = 1800, loss = 40.0680
t = 1900,

t = 100, loss = 39.0102
t = 200, loss = 40.9644
t = 300, loss = 38.8631
t = 400, loss = 38.6263
t = 500, loss = 40.0835
t = 600, loss = 38.6525
t = 700, loss = 40.3276
t = 800, loss = 39.6585
t = 900, loss = 40.4101
t = 1000, loss = 39.1349
t = 1100, loss = 40.2075
t = 1200, loss = 39.8682
t = 1300, loss = 41.6815
t = 1400, loss = 40.9978
t = 1500, loss = 39.0390
t = 1600, loss = 39.3296
t = 1700, loss = 40.4730
t = 1800, loss = 41.7843
t = 1900, loss = 39.2286
t = 2000, loss = 40.5266
Got 18636 / 20541 correct (90.73)
Starting epoch 122 / 200
t = 100, loss = 39.7108
t = 200, loss = 38.9613
t = 300, loss = 39.1916
t = 400, loss = 39.6607
t = 500, loss = 40.0582
t = 600, loss = 40.8275
t = 700, loss = 39.1772
t = 800, loss = 41.2368
t = 900, loss = 40.8864
t = 1000, loss = 40.0738
t = 1100, loss = 40.4931
t = 1200, loss = 38.7638
t = 1300, loss = 39.6646
t = 1400, loss = 37.7618
t = 1500, loss = 39.5184
t = 1600, loss = 40.3356
t = 1700, loss = 40.3156
t = 1800, loss = 40.8715
t = 1900,

In [12]:
def check_accuracy(model, loader):
    '''
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')  
    '''
    num_correct = 0
    num_samples = 0
    model.eval() # Put the model in test mode (the opposite of model.train(), essentially)
    for t, sample in enumerate(loader):
        
        x_var = Variable(sample['image'].float().cuda())
        y_var = Variable(sample['label'].long().cuda())
        
        y_var=y_var.cpu()
        scores = model(x_var)
        _, preds = scores.data.cpu().max(1)
        #print(preds)
        #print(y_var)
        num_correct += (preds.numpy() == y_var.numpy()).sum()
        num_samples += preds.size(0)
    acc = float(num_correct) / num_samples
    print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

In [ ]:

check_accuracy(resnet_binary, as_dataset_batches)

In [ ]:
torch.save(resnet_binary.state_dict(), "act4.h5")